In [1]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

# from config import db_password

import time

In [2]:
#  Add the clean movie function that takes in the argument, "movie".
def clean_movie(movie):

    
    
    

    return movie

In [6]:
# 1 Add the function that takes in three arguments;
# Wikipedia data, Kaggle metadata, and MovieLens rating data (from Kaggle)

def extract_transform_load(wiki, kaggle, rating):
    # Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    ratings = pd.read_csv(rating)
    kaggle_metadata = pd.read_csv(kaggle, low_memory=False)

    # Open and read the Wikipedia data JSON file.
    file = open(wiki)
    wiki_movies_raw = json.load(file)
    
    # Write a list comprehension to filter out TV shows.
    wiki_movies_raw = [data for data in wiki_movies_raw if 'Television series' not in data.keys()]

    # Write a list comprehension to iterate through the cleaned wiki movies list
    # and call the clean_movie function on each movie.
    cleaned_movies = [clean_movie(movie) for movie in wiki_movies_raw]

    # Read in the cleaned movies list from Step 4 as a DataFrame.
    wiki_movies_df = pd.DataFrame(cleaned_movies)


    # Write a try-except block to catch errors while extracting the IMDb ID using a regular expression string and
    #  dropping any imdb_id duplicates. If there is an error, capture and print the exception.
    try:
        unique_imdb = []
        wiki_movies = []
        for data in cleaned_movies:
            if 'imdb_link' in data.keys():
                imdb_id = re.findall(r'(tt\d{7})', data['imdb_link'])[0]
                if imdb_id not in unique_imdb:
                    wiki_movies.append(data)
                    unique_imdb.append(imdb_id)
    except:
        print('Error Occurs while removing duplicates from data using IMDB')

    #  Write a list comprehension to keep the columns that don't have null values from the wiki_movies_df DataFrame.
    wiki_movies_df = pd.DataFrame(wiki_movies)
    wiki_movies_df['imdb_link'] = unique_imdb
    wiki_movies_df = wiki_movies_df[[col for col in wiki_movies_df.columns if not wiki_movies_df[col].isnull().values.all()]]


    # Create a variable that will hold the non-null values from the “Box office” column.
    wiki_movies_df = wiki_movies_df[wiki_movies_df['Box office'].notnull()]
    box_office = wiki_movies_df['Box office'].values
    box_office = [data for data in box_office if isinstance(data, str)]
    
    # Convert the box office data created in Step 8 to string values using the lambda and join functions.
    box_office_func = lambda x:' '.join(x)
    box_office_data = box_office_func(box_office)

    # Write a regular expression to match the six elements of "form_one" of the box office data.
    form_one = r'(\d\d,\d\d\d | \d\d\d.\d\d)'
    # Write a regular expression to match the three elements of "form_two" of the box office data.
    form_two = r'(\d,\d | \d.\d)'

    # Add the parse_dollars function.
    def parse_dollars():
        pass
        
    # Clean the box office column in the wiki_movies_df DataFrame.
    for i in range(wiki_movies_df.shape[0]):
        try:
            val = wiki_movies_df.loc[i,'Box office']
            if not isinstance(val, str):
                val = val[0]
            try:
                wiki_movies_df.loc[i,'Box office'] = re.findall(form_one, val)[0]
            except:
                try:
                    wiki_movies_df.loc[i,'Box office'] = re.findall(form_two, val)[0]
                except:
                    wiki_movies_df.loc[i,'Box office'] = val
        except:
            pass
    
    # Clean the budget column in the wiki_movies_df DataFrame.
    for i in range(wiki_movies_df.shape[0]):
        try:
            val = wiki_movies_df.loc[i,'Budget']
            if not isinstance(val, str):
                val = val[0]
            wiki_movies_df.loc[i,'Budget'] = re.findall(r'([0-9]+)', val)[0]
        except:
            pass

    # Clean the release date column in the wiki_movies_df DataFrame.
    for i in range(wiki_movies_df.shape[0]):
        try:
            val = wiki_movies_df.loc[i,'Release date']
            if not isinstance(val, str):
                val = val[0]
            wiki_movies_df.loc[i,'Release date'] = val
        except:
            pass

    # Clean the running time column in the wiki_movies_df DataFrame.
    for i in range(wiki_movies_df.shape[0]):
        try:
            val = wiki_movies_df.loc[i,'Running time']
            if not isinstance(val, str):
                val = val[0]
            wiki_movies_df.loc[i,'Running time'] = re.findall(r'([0-9]+)', val)[0]
        except:
            pass
     
    # 2. Clean the Kaggle metadata.
    for i in range(kaggle_metadata.shape[0]):
        for col in kaggle_metadata.columns:
            val = kaggle_metadata.loc[i, col]
            if isinstance(val, dict):
                if 'name' in val.keys():
                    kaggle_metadata[i, col] = val['name']
                else:
                    kaggle_metadata[i, col] = np.nan
    

    # 3. Merged the two DataFrames into the movies DataFrame.
    movies_df =  pd.concat([kaggle_metadata, wiki_movies_df], axis=1)


    # 4. Drop unnecessary columns from the merged DataFrame.
    movies_df = movies_df[[col for col in movies_df.columns if not movies_df[col].isna().values.all()]]


    # 5. Add in the function to fill in the missing Kaggle data.
    def fill_missing_values(df, columns):
        df_columns = df.columns
        for col in columns:
            if col in df_columns:
                df[col] = df[col].fillna(method='ffill')
        return df
    
    movies_df = fill_missing_values(movies_df, kaggle_metadata.columns)

    # 6. Call the function in Step 5 with the DataFrame and columns as the arguments.
    movies_df = fill_missing_values(movies_df, wiki_movies_df.columns)

    # 7. Filter the movies DataFrame for specific columns.
    movies_df = movies_df[[col for col in movies_df.columns if not movies_df[col].isna().values.any()]]

    # 8. Rename the columns in the movies DataFrame.
    movies_df.rename(columns = {'belongs_to_collection':"collection"}, inplace = True)

    # 9. Transform and merge the ratings DataFrame.
    ratings.rename(columns = {'moviesId':"id"}, inplace = True)
    def double_rating(rating):
        return rating*2
    
    ratings['rating'] = ratings['rating'].transform(double_rating)
    movies_with_ratings_df = pd.concat([movies_df.loc[:100,:], ratings.loc[:100,:]], axis=1)
    
    return wiki_movies_df, movies_with_ratings_df, movies_df


In [7]:
# 10. Create the path to your file directory and variables for the three files.
file_dir = 'Resources'
# The Wikipedia data
wiki_file = f'{file_dir}/wikipedia_movies.json'
# The Kaggle metadata
kaggle_file = f'{file_dir}/movies_metadata.csv'
# The MovieLens rating data.
ratings_file = f'{file_dir}/ratings.csv'

In [8]:
# 11. Set the three variables equal to the function created in D1.
wiki_file, kaggle_file, ratings_file = extract_transform_load(wiki_file, kaggle_file, ratings_file)

In [9]:
# 12. Set the DataFrames from the return statement equal to the file names in Step 11. 
wiki_movies_df = wiki_file
movies_with_ratings_df = kaggle_file
movies_df = ratings_file

In [10]:
# 13. Check the wiki_movies_df DataFrame. 
wiki_movies_df.head()

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Hepburn,Literally,Cantonese,Chinese,Yiddish,Arabic,Romanized,Russian,Hebrew,Polish
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,tt0098987,The Adventures of Ford Fairlane,Renny Harlin,"[Steve Perry, Joel Silver]","[David Arnott, James Cappe, Daniel Waters]","[David Arnott, James Cappe]","[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,tt0098994,"After Dark, My Sweet",James Foley,"[Ric Kidney, Robert Redlin]","[James Foley, Robert Redlin]",NaN,"[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,tt0099005,Air America,Roger Spottiswoode,Daniel Melnick,"[John Eskow, Richard Rush]",NaN,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990,tt0099012,Alice,Woody Allen,Robert Greenhut,NaN,NaN,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990,tt0099018,Almost an Angel,John Cornell,John Cornell,NaN,NaN,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# 14. Check the movies_with_ratings_df DataFrame.
movies_with_ratings_df.head()

,adult,collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,Release date,Running time,Country,Language,Budget,Box office,userId,movieId,rating,timestamp
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,"July 11, 1990",102,United States,English,20,$21.4 million,1,110,2.0,1425941529
1,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",http://toystory.disney.com/toy-story,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,"May 17, 1990",114,United States,English,6,$2.7 million,1,147,9.0,1425942435
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",http://toystory.disney.com/toy-story,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,"August 10, 1990",113,United States,"[English, Lao]",35,"$57,718,089",1,858,10.0,1425941523
3,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",http://toystory.disney.com/toy-story,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,"December 25, 1990",106,United States,English,12,"$7,331,647",1,1221,10.0,1425941546
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",http://toystory.disney.com/toy-story,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,"December 19, 1990",95,US,English,25,"39,946",1,1246,10.0,1425941556


In [12]:
# 15. Check the movies_df DataFrame. 
movies_df.head()

,adult,collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,Cinematography,Edited by,Productioncompany,Distributed by,Release date,Running time,Country,Language,Budget,Box office
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,Oliver Wood,Michael Tronick,Silver Pictures,20th Century Fox,"July 11, 1990",102,United States,English,20,$21.4 million
1,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",http://toystory.disney.com/toy-story,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,Mark Plummer,Howard E. Smith,Avenue Pictures,Avenue Pictures,"May 17, 1990",114,United States,English,6,$2.7 million
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",http://toystory.disney.com/toy-story,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,Roger Deakins,"[John Bloom, Lois Freeman-Fox]","[Carolco Pictures, IndieProd Company]",TriStar Pictures,"August 10, 1990",113,United States,"[English, Lao]",35,"$57,718,089"
3,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",http://toystory.disney.com/toy-story,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,Carlo Di Palma,Susan E. Morse,"[Carolco Pictures, IndieProd Company]",Orion Pictures,"December 25, 1990",106,United States,English,12,"$7,331,647"
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",http://toystory.disney.com/toy-story,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,Russell Boyd,David Stiven,"[Carolco Pictures, IndieProd Company]",Paramount Pictures,"December 19, 1990",95,US,English,25,"39,946"
